In [3]:
import pandas as pd
import numpy as np

# Tratativa com dicionario

In [4]:
caminho_arquivo = 'Obesity.csv' # Usando o nome correto do seu CSV
df_original = pd.read_csv(caminho_arquivo)

# Dicionário de renomeação
mapa_colunas = {
    'Gender': 'genero',
    'Age': 'idade',
    'Height': 'altura_m',
    'Weight': 'peso_kg',
    'family_history': 'historia_familiar_sobrepeso',
    'FAVC': 'come_comida_calorica_freq',
    'FCVC': 'freq_consumo_vegetais',
    'NCP': 'num_refeicoes_principais',
    'CAEC': 'come_entre_refeicoes',
    'SMOKE': 'fumante',
    'CH2O': 'consumo_agua_litros',
    'SCC': 'monitora_calorias',
    'FAF': 'freq_atividade_fisica',
    'TUE': 'tempo_uso_dispositivos',
    'CALC': 'freq_consumo_alcool',
    'MTRANS': 'meio_transporte',
    'Obesity': 'nivel_obesidade'
}

# Renomear colunas
df = df_original.rename(columns=mapa_colunas)

# Ajuste de Tipos de Dados
df['idade'] = df['idade'].astype(int)
df['freq_consumo_vegetais'] = df['freq_consumo_vegetais'].astype(int)
df['num_refeicoes_principais'] = df['num_refeicoes_principais'].astype(int)
df['consumo_agua_litros'] = df['consumo_agua_litros'].astype(int)
df['freq_atividade_fisica'] = df['freq_atividade_fisica'].astype(int)
df['tempo_uso_dispositivos'] = df['tempo_uso_dispositivos'].astype(int)


In [5]:
print("DataFrame pronto para engenharia de características e pré-processamento.")
df.head()

DataFrame pronto para engenharia de características e pré-processamento.


,genero,idade,altura_m,peso_kg,historia_familiar_sobrepeso,come_comida_calorica_freq,freq_consumo_vegetais,num_refeicoes_principais,come_entre_refeicoes,fumante,consumo_agua_litros,monitora_calorias,freq_atividade_fisica,tempo_uso_dispositivos,freq_consumo_alcool,meio_transporte,nivel_obesidade
0,Female,21,1.62,64.0,yes,no,2,3,Sometimes,no,2,no,0,1,no,Public_Transportation,Normal_Weight
1,Female,21,1.52,56.0,yes,no,3,3,Sometimes,yes,3,yes,3,0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23,1.80,77.0,yes,no,2,3,Sometimes,no,2,no,2,1,Frequently,Public_Transportation,Normal_Weight
3,Male,27,1.80,87.0,no,no,3,3,Sometimes,no,2,no,2,0,Frequently,Walking,Overweight_Level_I
4,Male,22,1.78,89.8,no,no,2,1,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Overweight_Level_II


# CÉLULA 3: ENGENHARIA DE CARACTERÍSTICAS

# Criar a coluna 'imc' (Índice de Massa Corporal)
# Fórmula: peso (kg) / (altura (m))^2

In [6]:
df['imc'] = (df['peso_kg'] / (df['altura_m'] ** 2)).round(2)

# Exibir as primeiras linhas com a nova coluna para verificação
print("DataFrame com a nova característica 'imc':")
df[['altura_m', 'peso_kg', 'imc', 'nivel_obesidade']].head()

DataFrame com a nova característica 'imc':


,altura_m,peso_kg,imc,nivel_obesidade
0,1.62,64.0,24.39,Normal_Weight
1,1.52,56.0,24.24,Normal_Weight
2,1.80,77.0,23.77,Normal_Weight
3,1.80,87.0,26.85,Overweight_Level_I
4,1.78,89.8,28.34,Overweight_Level_II


# CÉLULA 4: SEPARAÇÃO DE VARIÁVEIS PREDITORAS (X) E VARIÁVEL ALVO (y)

In [7]:
# 'X' contém todas as colunas, exceto o nosso alvo 'nivel_obesidade'
X = df.drop('nivel_obesidade', axis=1)

# 'y' contém apenas a nossa coluna alvo
y = df['nivel_obesidade']

# Exibir as dimensões dos dataframes resultantes para verificação
print(f"Dimensões de X (variáveis preditoras): {X.shape}")
print(f"Dimensões de y (variável alvo): {y.shape}")

Dimensões de X (variáveis preditoras): (2111, 17)
Dimensões de y (variável alvo): (2111,)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

# 1. Separar dados de Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Definir quais colunas são categóricas e quais são numéricas
colunas_categoricas = [
    'genero', 'historia_familiar_sobrepeso', 'come_comida_calorica_freq',
    'come_entre_refeicoes', 'fumante', 'monitora_calorias',
    'freq_consumo_alcool', 'meio_transporte'
]

# As numéricas são todas as outras (incluindo o 'imc' que você criou)
colunas_numericas = [col for col in X.columns if col not in colunas_categoricas]

# 3. Criar o Preprocessador (Pipeline de transformação)
# Numéricas -> Padronização (StandardScaler)
# Categóricas -> Codificação OneHot (OneHotEncoder)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)
    ])

# 4. Criar o Pipeline Completo (Preprocessamento + Modelo)
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# 5. Treinar o Modelo
print("Treinando o modelo...")
model_pipeline.fit(X_train, y_train)

# 6. Avaliar o Modelo
y_pred = model_pipeline.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)

print(f"\n✅ Acurácia do modelo: {acuracia:.2%}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Treinando o modelo...

✅ Acurácia do modelo: 97.64%

Relatório de Classificação:
                     precision    recall  f1-score   support

Insufficient_Weight       0.98      0.98      0.98        56
      Normal_Weight       0.92      0.95      0.94        62
     Obesity_Type_I       0.99      1.00      0.99        78
    Obesity_Type_II       1.00      0.98      0.99        58
   Obesity_Type_III       1.00      1.00      1.00        63
 Overweight_Level_I       0.95      0.93      0.94        56
Overweight_Level_II       1.00      0.98      0.99        50

           accuracy                           0.98       423
          macro avg       0.98      0.98      0.98       423
       weighted avg       0.98      0.98      0.98       423



In [9]:
# Salvar o pipeline treinado em um arquivo .pkl
joblib.dump(model_pipeline, 'modelo_obesidade.pkl')
print("Modelo salvo com sucesso como 'modelo_obesidade.pkl'")

Modelo salvo com sucesso como 'modelo_obesidade.pkl'
